In [18]:
import pandas as pd
import numpy as np 
import argparse
import math
import sys

from scipy.stats import pearsonr
import matplotlib.pyplot as plt 

import smlmodule

from itertools import combinations

LIMIT = 0.95

__provmaps__ = {
    "bolzano_bozen": "bolzano",
    "bolzanobozen": "bolzano",
    "vibovalentia": "vibo_valentia",
    "laquila": "l_aquila",
    "laspezia": "la_spezia",
    "barlettaandriatrani": "bat",
    "ascolipiceno": "ascoli_piceno",
    "carboniaiglesias": "carbonia",
    "reggioemilia": "reggio_nell_emilia",
    "pesarourbino": "pesaro",
    "monzabrianza": "monza",
    "reggiocalabria": "reggio_di_calabria",
    "forlicesena": "forli",
    "massacarrara": "massa",
    "verbanocusioossola": "verbania",
    "verbano_cusio_ossola": "verbania",
    "massa_carrara": "massa",
    "monza_e_della_brianza": "monza",
    "pesaro_e_urbino": "pesaro",
    "forli__cesena": "forli",
    "bolzano_/_bozen": "bolzano",
    "barletta_andria_trani": "bat",
    "sud_sardegna": "carbonia",
    "forlì_cesena": "forli"
}

def filterprovname (inprov):
    low = inprov.lower()
    low = low.rstrip()
    low = low.lstrip()
    low = low.replace(" ", "_")
    low = low.replace("'", "_")
    low = low.replace("-", "_")

    return low

def normalize_provname (indata, provcolumn, verbose):

    dict_data = {}  
    for c in indata.columns:
        if verbose:
            print("  ", c)
        if c != provcolumn:
            dict_data[c] = []
    dict_data["prov"] = []

    for i, row in indata.iterrows():
        for c in indata.columns:    
            if c != provcolumn:
                dict_data[c].append(row[c])
            else:
                low = filterprovname(row[c])
                if low in __provmaps__:
                    low = __provmaps__[low]

                dict_data["prov"].append(low)

    #for v in dict_data:
    #    print(v, " ", len(dict_data[v]))

    data = pd.DataFrame.from_dict(dict_data)

    return data



In [19]:
verbose = False
paperpath = "particulate.csv"
agefeatspath = "provinceages.csv"
deprividxpath = "ID11_prov21.xlsx"
tabellecodicipath = "TabelleCodici.xlsx"
copernicopath = "name_region_province_statistics_2020.csv"

tc = pd.ExcelFile(tabellecodicipath)

idtoprov = {}
province = tc.parse("Codice Provincia")
for val in province[["Codice Provincia","Nome Provincia"]].values:
    if type(val[1]) != float:
        idtoprov[int(val[0])] = val[1]
        print(int(val[0]), val[1])

in_datapaper = pd.read_csv(paperpath, sep=";")
in_deprividx =  pd.ExcelFile(deprividxpath).parse("Foglio1")
in_agefeatures = pd.read_csv(agefeatspath)
in_agefeatures = in_agefeatures[in_agefeatures.Population2020 != 0.0]
in_copernico = pd.read_csv(copernicopath)

print("Paper data ")
datapaper = normalize_provname(in_datapaper, "Province", False)
print("Age features ")
agefeatures = normalize_provname(in_agefeatures, "Provincia", False)
print("Copernico data ") 
copernico = normalize_provname(in_copernico, "nome_ita", False)


1 TORINO
2 VERCELLI
3 NOVARA
4 CUNEO
5 ASTI
6 ALESSANDRIA
7 AOSTA
8 IMPERIA
9 SAVONA
10 GENOVA
11 LA SPEZIA
12 VARESE
13 COMO
14 SONDRIO
15 MILANO
16 BERGAMO
17 BRESCIA
18 PAVIA
19 CREMONA
20 MANTOVA
21 BOLZANO
22 TRENTO
23 VERONA
24 VICENZA
25 BELLUNO
26 TREVISO
27 VENEZIA
28 PADOVA
29 ROVIGO
30 UDINE
31 GORIZIA
32 TRIESTE
33 PIACENZA
34 PARMA
35 REGGIO NELL'EMILIA
36 MODENA
37 BOLOGNA
38 FERRARA
39 RAVENNA
40 FORLI'-CESENA
41 PESARO E URBINO
42 ANCONA
43 MACERATA
44 ASCOLI PICENO
45 MASSA CARRARA
46 LUCCA
47 PISTOIA
48 FIRENZE
49 LIVORNO
50 PISA
51 AREZZO
52 SIENA
53 GROSSETO
54 PERUGIA
55 TERNI
56 VITERBO
57 RIETI
58 ROMA
59 LATINA
60 FROSINONE
61 CASERTA
62 BENEVENTO
63 NAPOLI
64 AVELLINO
65 SALERNO
66 L'AQUILA
67 TERAMO
68 PESCARA
69 CHIETI
70 CAMPOBASSO
71 FOGGIA
72 BARI
73 TARANTO
74 BRINDISI
75 LECCE
76 POTENZA
77 MATERA
78 COSENZA
79 CATANZARO
80 REGGIO DI CALABRIA
81 TRAPANI
82 PALERMO
83 MESSINA
84 AGRIGENTO
85 CALTANISSETTA
86 ENNA
87 CATANIA
88 RAGUSA
89 SIRACUSA
90 SASSAR

In [20]:
dict_deprividx = {}
print("DrepivIdx name ")
for c in in_deprividx.columns:
    if verbose:
        print("   ", c)   
    dict_deprividx[c] = []
dict_deprividx["prov"] = []

for i, row in in_deprividx.iterrows():
    id = row["prov21"]
    prov = filterprovname(idtoprov[id])
    
    if prov in __provmaps__:
        prov = __provmaps__[prov]
    
    #print(id, prov)
    dict_deprividx["prov"].append(prov)
    for c in in_deprividx.columns:
        dict_deprividx[c].append(row[c])


deprividx = pd.DataFrame.from_dict(dict_deprividx)       

DrepivIdx name 


In [21]:
provincelist = list(set(list(datapaper["prov"].values)) & \
        set(list(deprividx["prov"].values)) & \
        set(list(agefeatures["prov"].values)) &
        set(list(copernico["prov"].values)))

print("Province list: ")
for i, p in enumerate(provincelist):
    print("  ", i+1, " ", p)

Province list: 
   1   verbania
   2   nuoro
   3   bat
   4   como
   5   agrigento
   6   biella
   7   catanzaro
   8   trento
   9   sondrio
   10   torino
   11   sassari
   12   caserta
   13   mantova
   14   asti
   15   ancona
   16   lodi
   17   matera
   18   oristano
   19   prato
   20   vercelli
   21   trapani
   22   venezia
   23   lecce
   24   pordenone
   25   bolzano
   26   vicenza
   27   pistoia
   28   ravenna
   29   pisa
   30   bergamo
   31   teramo
   32   latina
   33   siracusa
   34   l_aquila
   35   enna
   36   messina
   37   verona
   38   piacenza
   39   perugia
   40   ferrara
   41   isernia
   42   gorizia
   43   reggio_di_calabria
   44   varese
   45   milano
   46   fermo
   47   campobasso
   48   reggio_nell_emilia
   49   forli
   50   belluno
   51   macerata
   52   parma
   53   arezzo
   54   trieste
   55   chieti
   56   potenza
   57   pescara
   58   napoli
   59   ragusa
   60   massa
   61   grosseto
   62   ascoli_piceno
   

In [29]:
counter = 0

# TODO important  Double check period1 same as paper ? 
pollutantsnames = "avg_wco_period1_2020,"+\
        "avg_wnh3_period1_2020,"+\
        "avg_wnmvoc_period1_2020,"+\
        "avg_wno2_period1_2020,"+\
        "avg_wno_period1_2020,"+\
        "avg_wo3_period1_2020,"+\
        "avg_wpans_period1_2020,"+\
        "avg_wpm10_period1_2020,"+\
        "avg_wpm2p5_period1_2020,"+\
        "avg_wso2_period1_2020," +\
        "sum_wnh3_ex_q75_period1_2020," +\
        "sum_wnmvoc_ex_q75_period1_2020," +\
        "sum_wno2_ex_q75_period1_2020," +\
        "sum_wno_ex_q75_period1_2020," +\
        "sum_wpans_ex_q75_period1_2020," +\
        "sum_wpm10_ex_q75_period1_2020," +\
        "sum_wpm2p5_ex_q75_period1_2020," +\
        "sum_wo3_ex_q75_period1_2020," + \
        "sum_wco_ex_q75_period1_2020," + \
        "sum_wso2_ex_q75_period1_2020"

featurestobeused = "density," + \
        "commutersdensity," + \
        "depriv," + \
        "Ratio0200ver65," + \
        "sum_wpm10_ex_q75_period1_2020,"+\
        "sum_wpm2p5_ex_q75_period1_2020,"+\
        "sum_wno2_ex_q75_period1_2020,"+\
        "sum_wno_ex_q75_period1_2020,"+\
        "sum_wnh3_ex_q75_period1_2020,"+\
        "sum_wpans_ex_q75_period1_2020,"+\
        "sum_wnmvoc_ex_q75_period1_2020,"+\
        "sum_wo3_ex_q75_period1_2020,"+\
        "sum_wco_ex_q75_period1_2020,"+\
        "sum_wso2_ex_q75_period1_2020"

for prov in provincelist:
    cases = datapaper[datapaper["prov"] == prov]["Cases"].values[0]
    popolazione = datapaper[datapaper["prov"] == prov]["Population"].values[0]
    pop2 = agefeatures[agefeatures["prov"] == prov]["Population2020"].values[0]
    diff = 100.0*(math.fabs(popolazione-pop2)/(popolazione))

    if cases > 0.0 and diff < 5.0:
        counter += 1

ylogpropcasi = []
features_dict = {}

for fn in ("population", "density", "commutersdensity", "depriv", "lat", "Ratio0200ver65"):
    features_dict[fn] = np.zeros(counter, dtype="float64")

for fn in pollutantsnames.split(","):
    features_dict[fn] = np.zeros(counter, dtype="float64")
    
for idx, prov in enumerate(provincelist):

    cases = datapaper[datapaper["prov"] == prov]["Cases"].values[0]
    popolazione = datapaper[datapaper["prov"] == prov]["Population"].values[0]
    pop2 = agefeatures[agefeatures["prov"] == prov]["Population2020"].values[0]

    diff = 100.0*(math.fabs(popolazione-pop2)/(popolazione))
    
    ycasi = cases/popolazione

    if cases > 0.0 and diff < 5.0:
        ylogpropcasi = math.log(ycasi)

        selected = copernico[copernico["prov"] == prov]

        features_dict["population"][i] = popolation
        features_dict["density"][i] = \
                    datapaper[datapaper["prov"] == prov]["Density"].values[0]    
        features_dict["commutersdensity"][i] = \
                    datapaper[datapaper["prov"] == prov]["CommutersDensity"].values[0]       
        features_dict["lat"][i] = \
                    datapaper[datapaper["prov"] == prov]["Lat"].values[0]       
        features_dict["depriv"][i] = \
                    deprividx[deprividx["prov"] == prov]["ID_2011"].values[0]
        #print(idx, prov, agefeatures[agefeatures["prov"] == prov])
        features_dict["Ratio0200ver65"][i] = \
                    agefeatures[agefeatures["prov"] == prov]["Ratio0200ver65"].values[0]
